In [1]:
import numpy as np
import pandas as pd

In [2]:
# HYPERPARAMETER BY HANDS

In [3]:
data = pd.read_csv('../Evaluate Model (P13)/heart_disease.csv')

In [4]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
## Sklearn

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
clf = RandomForestClassifier()

In [9]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [10]:
# evaluate_function for reuse on classifier model, 
#Manual Evaluate 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [11]:
def evaluate_preds(y_true,y_preds):
    accuracy = accuracy_score(y_true,y_preds)
    precision = precision_score(y_true,y_preds)
    recall = recall_score(y_true,y_preds)
    f1 = f1_score(y_true,y_preds)
    data_dict = { 'Accuracy': round(accuracy,2),
                'Precision': round(precision,2),
                'Recall': round(recall,2),
                'f1':round(f1,2)}
    print(f'Accuracy:{accuracy*100:.2f}%')
    print(f'Precision:{precision:.2f}')
    print(f'Recall:{recall:.2f}')
    print(f'f1:{f1:.2f}')
    
    return data_dict

In [12]:
# Custom train_test_split

In [13]:
np.random.seed(1)

#shuffle
data_shuffle = data.sample(frac=1)

#split
featured_shuffle = data_shuffle.drop('target',axis=1)
target_shuffle = data_shuffle['target']

#split validation and test 
train_split_s = round(0.7* len(data_shuffle))

valid_split_s = round(train_split_s + 0.15 *len(data_shuffle))

x_train,y_train = featured_shuffle[:train_split_s], target_shuffle[:train_split_s]

x_valid,y_valid = featured_shuffle[train_split_s:valid_split_s],target_shuffle[train_split_s:valid_split_s]

x_test,y_test = featured_shuffle[valid_split_s:],target_shuffle[valid_split_s]

In [14]:
len(x_train), len(x_valid), len(x_test)

(212, 45, 46)

In [15]:
#fit
clf.fit(x_train,y_train)

RandomForestClassifier()

In [16]:
#prediction

In [17]:
y_preds_valid = clf.predict(x_valid) #baseline Prediction

In [18]:
baseline_metrics = evaluate_preds(y_valid,y_preds_valid)

Accuracy:93.33%
Precision:0.96
Recall:0.93
f1:0.94


In [23]:
clf.get_params() #this is default settings of hyperparameter.Chnaging it by hand one could adjut the improvement of the model

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [20]:
##########################
#
# Chossing hyperparater using randomize cv with scikit
#
############################

In [27]:
from sklearn.model_selection import RandomizedSearchCV,train_test_split

In [33]:
#creating  a dict for the hyperparameter one needs  to adjust
# Read the scikit documentation for the values to use in hyperparameter
hype_dict = {
    'n_estimators':[100,200,500,1200],
    'max_depth':[None,5,10,20,30],
    'max_features':['auto','sqrt'],
    'min_samples_split':[2,4,6],
    'min_samples_leaf': [1,2,4]
}

In [36]:
np.random.seed(33)
#shuffle
data_shuffle2 = data.sample(frac=1)
#split
featured_shuf2 = data_shuffle2.drop('target',axis=1)
target_shuf2 = data_shuffle2['target']

x2_train,x2_test,y2_train,y2_test = train_test_split(featured_shuf2,target_shuf2,test_size=0.2)

clf2 = RandomForestClassifier()

#randomize search cv
rs_clf2 = RandomizedSearchCV(estimator=clf2, param_distributions=hype_dict, n_iter=10,cv=5,verbose=2)

rs_clf2.fit(x2_train,y2_train)



Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, total=   1.8s
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, total=   1.8s
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20 
[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, total=   1.9s
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20 
[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, total=   1.8s
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20 
[CV]  n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, total=   1.8s
[CV] n_estimators=100, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=None 
[CV]  n_estimators=100, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=None, total=   0.1s
[CV] n_estimators=100, min_samples_split=

[CV]  n_estimators=1200, min_samples_split=6, min_samples_leaf=2, max_features=auto, max_depth=20, total=   1.9s
[CV] n_estimators=1200, min_samples_split=6, min_samples_leaf=2, max_features=auto, max_depth=20 
[CV]  n_estimators=1200, min_samples_split=6, min_samples_leaf=2, max_features=auto, max_depth=20, total=   1.8s
[CV] n_estimators=1200, min_samples_split=6, min_samples_leaf=2, max_features=auto, max_depth=20 
[CV]  n_estimators=1200, min_samples_split=6, min_samples_leaf=2, max_features=auto, max_depth=20, total=   1.9s
[CV] n_estimators=1200, min_samples_split=6, min_samples_leaf=2, max_features=auto, max_depth=20 
[CV]  n_estimators=1200, min_samples_split=6, min_samples_leaf=2, max_features=auto, max_depth=20, total=   1.9s
[CV] n_estimators=100, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=5 
[CV]  n_estimators=100, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=5, total=   0.2s
[CV] n_estimators=100, min_samples_split=2, min

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   31.0s finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [None, 5, 10, 20, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 4, 6],
                                        'n_estimators': [100, 200, 500, 1200]},
                   verbose=2)

In [38]:
rs_clf2.best_params_

{'n_estimators': 100,
 'min_samples_split': 6,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': None}

In [39]:
## Make prediction with the best_prarams_

In [40]:
rs_y_preds = rs_clf2.predict(x2_test)

#Evaluate the metrics
rs_metrics = evaluate_preds(y2_test,rs_y_preds)

Accuracy:85.25%
Precision:0.86
Recall:0.93
f1:0.89
